<a href="https://colab.research.google.com/github/Gedeon-m-gedus/Image_Processing/blob/master/Pruning_quantiztion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install torchsummaryX

In [10]:
import torch
from torch import nn
import numpy as np
import torch.nn.utils.prune as prune
import torchvision.models as models
import torch.nn.functional as F
from torchsummaryX import summary
from torchvision import datasets, transforms as T
import os

In [9]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

set_seed(42)

In [12]:
# Colab's file access feature
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [14]:
save_dir = "/content/gdrive/My Drive/AMMI_project/object_detector"
handle_dirs(save_dir)

In [3]:
model = models.resnext50_32x4d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


In [15]:
model.state_dict();
PATH = save_dir + '/Resnet50_full_precission.pth'
torch.save(model.state_dict(), PATH)

In [5]:
summary(model,torch.zeros([1, 3, 224, 224]))

                                             Kernel Shape       Output Shape  \
Layer                                                                          
0_conv1                                     [3, 64, 7, 7]  [1, 64, 112, 112]   
1_bn1                                                [64]  [1, 64, 112, 112]   
2_relu                                                  -  [1, 64, 112, 112]   
3_maxpool                                               -    [1, 64, 56, 56]   
4_layer1.0.Conv2d_conv1                   [64, 128, 1, 1]   [1, 128, 56, 56]   
5_layer1.0.BatchNorm2d_bn1                          [128]   [1, 128, 56, 56]   
6_layer1.0.ReLU_relu                                    -   [1, 128, 56, 56]   
7_layer1.0.Conv2d_conv2                    [4, 128, 3, 3]   [1, 128, 56, 56]   
8_layer1.0.BatchNorm2d_bn2                          [128]   [1, 128, 56, 56]   
9_layer1.0.ReLU_relu                                    -   [1, 128, 56, 56]   
10_layer1.0.Conv2d_conv3                

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[3, 64, 7, 7]","[1, 64, 112, 112]",9408.0,118013952.0
1_bn1,[64],"[1, 64, 112, 112]",128.0,64.0
2_relu,-,"[1, 64, 112, 112]",NaN,NaN
3_maxpool,-,"[1, 64, 56, 56]",NaN,NaN
4_layer1.0.Conv2d_conv1,"[64, 128, 1, 1]","[1, 128, 56, 56]",8192.0,25690112.0
...,...,...,...,...
153_layer4.2.Conv2d_conv3,"[1024, 2048, 1, 1]","[1, 2048, 7, 7]",2097152.0,102760448.0
154_layer4.2.BatchNorm2d_bn3,[2048],"[1, 2048, 7, 7]",4096.0,2048.0
155_layer4.2.ReLU_relu,-,"[1, 2048, 7, 7]",NaN,NaN


In [21]:
#model.named_modules

In [29]:
new_model = model
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['conv1.weight_mask', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight_mask', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight_mask', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight_mask', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight_mask', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight_mask', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight_mask', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer1.1.conv3.weight_mask', 'layer1.1.bn3.running_mean', 'layer1.1.bn3.running_var', 'layer1.1.bn3.num_batches_tracked', 'layer1

In [30]:
new_model.state_dict();
PATH = save_dir + '/Resnet50_pruned.pth'
torch.save(new_model.state_dict(), PATH)

In [37]:
new_model#.layer1.state_dict().keys()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1